**MediMateAI** is a cutting-edge tool leveraging artificial intelligence to assist users in extracting vital information from their prescriptions. Designed to enhance medication management, it offers comprehensive details on potential side effects, alerts users to conflicts between current and previous prescriptions, and serves as a reminder for consistent medication intake.

**Disclaimer:**

Please be aware that MediMateAI is currently in beta. While it provides valuable insights, it is not a substitute for professional medical advice from your doctor. Moreover, it is not yet ready for widespread use and should not be relied upon for predictive purposes.

**Features:**

* Prescription Analysis:
        Users can upload a photo of their prescription.
        Receive a detailed report on medication side effects, potential conflicts, and dosage instructions.

  Users can upload a photo of their prescription.

* Medication Reminder:
      Provides users with reminders to take their medication as prescribed.
      Aids in maintaining a consistent medication schedule.

* Prescription History and Tracking:
        
        Allows users to view their prescription history.
        Enables tracking of medication intake over time.

**Development Stack:**

* Machine Learning Model:
        Utilized Google Cloud's Vertex AI to generate response
        Leveraged openAi embedding to obtain vectors from prescription images.

* Database:
      Implemented ZILLIZ as a vector database to store embedding vectors.

* Language Model Integration:
        Incorporated Langchin for designing and integrating the LLM (Large Language Model) service.

* Web Application:
        Developed the user interface using Streamlit to build an interactive and user-friendly web app.

* Model Evaluation:
        Employed trulens to evaluate the results generated by the machine learning model.

MediMateAI Notebook aims to revolutionize medication management by providing users with a seamless experience for prescription analysis, reminders, and tracking, all powered by advanced artificial intelligence. Please keep in mind the beta status and the importance of consulting with your healthcare professional for personalized medical advice.
Is this

In [1]:
! pip install -q google-cloud-aiplatform==1.36.3 litellm==1.11.1 trulens_eval==0.18.3 langchain==0.0.347
!pip install -q google-cloud-vision
!pip install -q --upgrade google-cloud-documentai
!pip install -q faiss-cpu tiktoken
!pip install huggingface -q
!pip install openai -q
!pip install pymilvus -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 627.0/627.0 kB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.5/221.5 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 929.3/929.3 kB 55.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.7/194.7 kB 12.2 MB

In [48]:
# FIX THIS ERROR : ContextualVersionConflict: (pyarrow 9.0.0 (/usr/local/lib/python3.10/dist-packages), Requirement.parse('pyarrow>=12.0.0'), {'pymilvus'})
!pip install pyarrow>=12.0.0
!pip install --upgrade ipython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.2/808.2 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: ipython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipython-7.34.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 8.18.1 which is incompatible.


In [86]:
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import json
from IPython.display import JSON, Markdown, display
import functools
import time
import math
from getpass import getpass
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm



from google.oauth2 import service_account
from google.api_core.client_options import ClientOptions
from google.cloud import documentai
from google.cloud import aiplatform
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials

from langchain.chains import LLMChain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.llms import VertexAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.embeddings import VertexAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from vertexai.language_models import TextEmbeddingModel
from langchain.vectorstores import Milvus
from vertexai.language_models import TextEmbeddingModel
from sklearn.metrics.pairwise import cosine_similarity
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings

from trulens_eval import TruChain, Feedback, Tru, LiteLLM, Select, feedback, TruCustomApp
from trulens_eval.feedback import Groundedness

from trulens_eval.tru_custom_app import instrument



# Setting Environment Keys

In [2]:
OPENAI_API_KEY = getpass()


··········


In [3]:
#google service_account file name
GOOGLE_APPLICATION_CREDENTIALS = getpass()


··········


In [4]:
#Google document AI
PROCESSOR_ID = getpass()


··········


In [5]:
PROCESSOR_PROJECT_ID = getpass()


··········


In [6]:
PROCESSOR_LOCATION = getpass()


··········


In [7]:
ZILLIZ_CLOUD_URI = getpass()


··········


In [8]:
ZILLIZ_CLOUD_API_KEY = getpass()

··········


## Init Trulens

**Load GCP credentials**

In [9]:
credentials = service_account.Credentials.from_service_account_file(os.path.join(os.getcwd(), GOOGLE_APPLICATION_CREDENTIALS))

In [10]:
aiplatform.init(credentials=credentials, project=credentials.project_id)

### Preprocessing Dataset

In [11]:
DATASET_PATH = os.path.join(os.getcwd(),'drugs_side_effects_drugs_com.csv')
DATASET_PATH

'/content/drugs_side_effects_drugs_com.csv'

In [12]:
df = pd.read_csv(DATASET_PATH,  error_bad_lines=False)

<ipython-input-12-d2f79e1b7e88>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(DATASET_PATH,  error_bad_lines=False)
Skipping line 156: expected 17 fields, saw 27



In [13]:
df.head(2)

,drug_name,medical_condition,side_effects,generic_name,drug_classes,brand_names,activity,rx_otc,pregnancy_category,csa,alcohol,related_drugs,medical_condition_description,rating,no_of_reviews,drug_link,medical_condition_url
0,doxycycline,Acne,"(hives, difficult breathing, swelling in your ...",doxycycline,"Miscellaneous antimalarials, Tetracyclines","Acticlate, Adoxa CK, Adoxa Pak, Adoxa TT, Alod...",87%,Rx,D,N,X,amoxicillin: https://www.drugs.com/amoxicillin...,Acne Other names: Acne Vulgaris; Blackheads; B...,6.8,760.0,https://www.drugs.com/doxycycline.html,https://www.drugs.com/condition/acne.html
1,spironolactone,Acne,hives ; difficulty breathing; swelling of your...,spironolactone,"Aldosterone receptor antagonists, Potassium-sp...","Aldactone, CaroSpir",82%,Rx,C,N,X,amlodipine: https://www.drugs.com/amlodipine.h...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.2,449.0,https://www.drugs.com/spironolactone.html,https://www.drugs.com/condition/acne.html


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3145 entries, 0 to 3144
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   drug_name                      3145 non-null   object 
 1   medical_condition              3145 non-null   object 
 2   side_effects                   3017 non-null   object 
 3   generic_name                   3101 non-null   object 
 4   drug_classes                   3061 non-null   object 
 5   brand_names                    1866 non-null   object 
 6   activity                       3145 non-null   object 
 7   rx_otc                         3144 non-null   object 
 8   pregnancy_category             2894 non-null   object 
 9   csa                            3145 non-null   object 
 10  alcohol                        1498 non-null   object 
 11  related_drugs                  1576 non-null   object 
 12  medical_condition_description  3145 non-null   o

In [15]:
enhanced_df = df[['drug_name', 'medical_condition', 'side_effects', 'generic_name', 'drug_classes' ,	'brand_names' ]]


In [16]:
## Drop NUll
enhanced_df.dropna(inplace=True)

<ipython-input-16-f18f0fa4594f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enhanced_df.dropna(inplace=True)


In [17]:
#drop dublicate if exist

enhanced_df.drop_duplicates(inplace=True)


<ipython-input-17-f754bb1f8c77>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enhanced_df.drop_duplicates(inplace=True)


In [18]:
enhanced_df.head()

,drug_name,medical_condition,side_effects,generic_name,drug_classes,brand_names
0,doxycycline,Acne,"(hives, difficult breathing, swelling in your ...",doxycycline,"Miscellaneous antimalarials, Tetracyclines","Acticlate, Adoxa CK, Adoxa Pak, Adoxa TT, Alod..."
1,spironolactone,Acne,hives ; difficulty breathing; swelling of your...,spironolactone,"Aldosterone receptor antagonists, Potassium-sp...","Aldactone, CaroSpir"
2,minocycline,Acne,"skin rash, fever, swollen glands, flu-like sym...",minocycline,Tetracyclines,"Dynacin, Minocin, Minolira, Solodyn, Ximino, V..."
4,clindamycin,Acne,hives ; difficult breathing; swelling of your ...,clindamycin topical,"Topical acne agents, Vaginal anti-infectives","Cleocin T, Clindacin ETZ, Clindacin P, Clindag..."
5,Aldactone,Acne,hives ; difficulty breathing; swelling of your...,spironolactone,"Aldosterone receptor antagonists, Potassium-sp...",CaroSpir


In [19]:
ENHANCED_DF_PATH= "enhanced_med.csv"
enhanced_df.to_csv(ENHANCED_DF_PATH)

## Embedded data
In this step, we will convert data from a text format into vectors, capturing its semantic meaning.

# Using Vertex AI Embeddings with Quota Limit Workaround

If you intend to utilize Vertex AI Embeddings and encounter quota limitations, the following code provides a workaround. However, At final implentation we used OpenAI

In [20]:
def generate_batches(sentences, batch_size = 200):
    for i in range(0, len(sentences), batch_size):
        yield sentences[i : i + batch_size]

def encode_texts_to_embeddings(sentences):
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
    try:
        embeddings = model.get_embeddings(sentences)
        return [embedding.values for embedding in embeddings]
    except Exception:
        return [None for _ in range(len(sentences))]

def encode_text_to_embedding_batched(sentences, api_calls_per_second = 0.33, batch_size = 5):

    embeddings_list = []

    batches = generate_batches(sentences, batch_size)

    seconds_per_job = 1 / api_calls_per_second

    with ThreadPoolExecutor() as executor:
        futures = []
        for batch in tqdm(
            batches, total = math.ceil(len(sentences) / batch_size), position=0
        ):
            futures.append(
                executor.submit(functools.partial(encode_texts_to_embeddings), batch)
            )
            time.sleep(seconds_per_job)

        for future in futures:
            embeddings_list.extend(future.result())

    is_successful = [
        embedding is not None for sentence, embedding in zip(sentences, embeddings_list)
    ]
    if isinstance(embeddings_list[0], list):
        embeddings_list_successful = [embedding for embedding in embeddings_list if embedding is not None]
    return embeddings_list


In [21]:
# Vertex AI prediction quota is 20 / minute
from typing import List
def rate_limit(max_per_minute: int = 20):
    period = 60 / max_per_minute
    print('Waiting')
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print('.', end='')
            time.sleep(sleep_time)

class RateLimitedVertexAIEmbeddings(VertexAIEmbeddings):

    def embed_documents(self, texts: List[str], batch_size: int = 5) -> List[List[float]]:
        return encode_text_to_embedding_batched(texts)



In [22]:
#embeddings = RateLimitedVertexAIEmbeddings(disallowed_special=())
embeddings = OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY)

In [23]:
loader = CSVLoader(file_path=ENHANCED_DF_PATH,source_column="drug_name", encoding="utf-8", csv_args={
                'delimiter': ','})
data = loader.load()


In [30]:
data[0].page_content

': 0\ndrug_name: doxycycline\nmedical_condition: Acne\nside_effects: (hives, difficult breathing, swelling in your face or throat) or a severe skin reaction (fever, sore throat, burning in your eyes, skin pain, red or purple skin rash that spreads and causes blistering and peeling). Seek medical treatment if you have a serious drug reaction that can affect many parts of your body. Symptoms may include: skin rash, fever, swollen glands, flu-like symptoms, muscle aches, severe weakness, unusual bruising, or yellowing of your skin or eyes. This reaction may occur several weeks after you began using doxycycline. Doxycycline may cause serious side effects. Call your doctor at once if you have: severe stomach pain, diarrhea that is watery or bloody; throat irritation, trouble swallowing; chest pain, irregular heart rhythm, feeling short of breath; little or no urination; low white blood cell counts - fever, chills, swollen glands, body aches, weakness, pale skin, easy bruising or bleeding; s

In [25]:
## IN case you want create new collection
# vector_db = Milvus.from_documents(
#     data,
#     embeddings,
#     collection_name="medimate",
#     connection_args={
#         "uri": ZILLIZ_CLOUD_URI,
#         "token": ZILLIZ_CLOUD_API_KEY,
#         "secure": True,
#     },
# )

In [26]:
## we load existing collection
vector_db =Milvus(
    embeddings,
    collection_name="medimate",
    connection_args={
        "uri": ZILLIZ_CLOUD_URI,
        "token": ZILLIZ_CLOUD_API_KEY,
        "secure": True,
    },
)

In [28]:
res = vector_db.similarity_search("What is the side effect of doxycycline?")

In [31]:
res[0].page_content

'drug_name: doxycycline\nside_effects: (hives, difficult breathing, swelling in your face or throat) or a severe skin reaction (fever, sore throat, burning in your eyes, skin pain, red or purple skin rash that spreads and causes blistering and peeling). Seek medical treatment if you have a serious drug reaction that can affect many parts of your body. Symptoms may include: skin rash, fever, swollen glands, flu-like symptoms, muscle aches, severe weakness, unusual bruising, or yellowing of your skin or eyes. This reaction may occur several weeks after you began using doxycycline. Doxycycline may cause serious side effects. Call your doctor at once if you have: severe stomach pain, diarrhea that is watery or bloody; throat irritation, trouble swallowing; chest pain, irregular heart rhythm, feeling short of breath; little or no urination; low white blood cell counts - fever, chills, swollen glands, body aches, weakness, pale skin, easy bruising or bleeding; severe headaches, ringing in yo

**OCR Prescription**

In [34]:
def process_prescription(
    project_id: str,
    location: str,
    processor_id: str,
    file_path: str,
    mime_type: str,
    field_mask = None,
    processor_version_id= None,
) -> str:
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts, credentials= credentials)

    if processor_version_id:
        name = client.processor_version_path(
            project_id, location, processor_id, processor_version_id
        )
    else:

        name = client.processor_path(project_id, location, processor_id)

    # Read the file into memory
    with open(file_path, "rb") as image:
        image_content = image.read()

    # Load binary data
    raw_document = documentai.RawDocument(content=image_content, mime_type=mime_type)
    process_options = documentai.ProcessOptions(
        individual_page_selector=documentai.ProcessOptions.IndividualPageSelector(
            pages=[1]
        )
    )

    request = documentai.ProcessRequest(
        name=name,
        raw_document=raw_document,
        field_mask=field_mask,
        process_options=process_options,
    )

    result = client.process_document(request=request)
    document = result.document


    return document.text


In [36]:
prescription_img_path = "images.jpeg"

In [37]:
prescription = process_prescription(
  project_id=PROCESSOR_PROJECT_ID,
  location=PROCESSOR_LOCATION,
  processor_id=PROCESSOR_ID,
  file_path=prescription_img_path,
  mime_type = 'image/jpeg'
)
print(prescription)

دكتـور
معـاذ عطيتـو
طبيب وجراح العيون والمياه البيضاء والليزك
جامعـة الأزهـر - مستشفى الأقصر الدولي
الإسم : أشرف أدهـم سـيف
السـن 11
2
Moaz Ateto
التاريخ : ٢٤ / ۱۱ / ۲۰۲۳م
Lubricant Eye Drops E.D.
قطرة 3 مرات يوميا لمدة أسبوعين
RI
R/Analgesic Drug TAB.
قرص بعد الأكل مرتين يوميـا
لمدة أسبوع " ولا يكرر
RI Antibiotic Eye Ointment. E.O.
مرهــم مسـاء المـدة أسبوع )



**Generate JSON Response Containing Medicine Name and Recommended Dosage**

In this section, we will discuss how to use extracted text to make the JSON response that the MediMateAI app needs, which contains crucial information about prescribed medicines, including the medicine name and the recommended dosage.
JSON Response Structure

The JSON response is structured to provide clear and organized data for each prescribed medicine. It consists of two key components:

    Medicine Name: Clearly identifies the name of the prescribed medicine.
    Dosage Information: Specifies the recommended dosage and the frequency at which the medicine should be taken.
    Patient name
    Doctor name
    Date



In [38]:
medical_prescription_template = """\
    For the following text, extract the following information:
    - medicine name : list of medicine names that are mentioned in the text
    - taken number : number of times a day the medicine should be taken for each medicine name
    - patient name : name of the patient
    - doctor name : name of the doctor
    - date : date of the prescription
    text: {text}
    << FORMATTING >>
    Return a markdown code snippet with a Json  object formatted like this:
    ```json
        {{
        "medicine name": taken_number,
        "patient": "patient_name",
        "doctor": "doctor_name",
        "date": "date",
        }}
        ```
    """


In [39]:
full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=medical_prescription_template,
        input_variables=["text"],
    )
)


In [40]:
chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])


In [41]:
chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

llm = VertexAI()

chain = LLMChain(llm=llm, prompt=chat_prompt_template, verbose=True)



In [42]:
llm_response = chain(prescription)

display(llm_response)



> Entering new LLMChain chain...
Prompt after formatting:
Human:     For the following text, extract the following information:
    - medicine name : list of medicine names that are mentioned in the text
    - taken number : number of times a day the medicine should be taken for each medicine name
    - patient name : name of the patient
    - doctor name : name of the doctor
    - date : date of the prescription
    text: دكتـور
معـاذ عطيتـو
طبيب وجراح العيون والمياه البيضاء والليزك
جامعـة الأزهـر - مستشفى الأقصر الدولي
الإسم : أشرف أدهـم سـيف
السـن 11
2
Moaz Ateto
التاريخ : ٢٤ / ۱۱ / ۲۰۲۳م
Lubricant Eye Drops E.D.
قطرة 3 مرات يوميا لمدة أسبوعين
RI
R/Analgesic Drug TAB.
قرص بعد الأكل مرتين يوميـا
لمدة أسبوع " ولا يكرر
RI Antibiotic Eye Ointment. E.O.
مرهــم مسـاء المـدة أسبوع )

    << FORMATTING >>
    Return a markdown code snippet with a Json  object formatted like this:
    ```json
        {
        "medicine name": taken_number,
        "patient": "patient_name",
        "docto

{'text': ' ```json\n{\n  "medicine name": {\n    "Lubricant Eye Drops E.D.": "3 times a day for two weeks",\n    "Analgesic Drug TAB.": "once after meal twice a day for a week",\n    "Antibiotic Eye Ointment. E.O.": "once at night for a week"\n  },\n  "patient": "أشرف أدهـم سـيف",\n  "doctor": "معـاذ عطيتـو",\n  "date": "٢٤ / ۱۱ / ۲۰۲۳م"\n}\n```'}

In [43]:
llm_response

{'text': ' ```json\n{\n  "medicine name": {\n    "Lubricant Eye Drops E.D.": "3 times a day for two weeks",\n    "Analgesic Drug TAB.": "once after meal twice a day for a week",\n    "Antibiotic Eye Ointment. E.O.": "once at night for a week"\n  },\n  "patient": "أشرف أدهـم سـيف",\n  "doctor": "معـاذ عطيتـو",\n  "date": "٢٤ / ۱۱ / ۲۰۲۳م"\n}\n```'}

In [44]:
uncleaned_data = llm_response["text"].split('\n')

In [45]:
uncleaned_data[2:-2]

['  "medicine name": {',
 '    "Lubricant Eye Drops E.D.": "3 times a day for two weeks",',
 '    "Analgesic Drug TAB.": "once after meal twice a day for a week",',
 '    "Antibiotic Eye Ointment. E.O.": "once at night for a week"',
 '  },',
 '  "patient": "أشرف أدهـم سـيف",',
 '  "doctor": "معـاذ عطيتـو",',
 '  "date": "٢٤ / ۱۱ / ۲۰۲۳م"']

In [46]:
def convert_text_to_dict(text):
    try:
        text_str = ' '.join(text)

        dictionary = eval('{' + text_str + '}')
        return dictionary
    except Exception as e:
        print(f"Error converting text to dictionary: {e}")
        return None

input_text = uncleaned_data[2:-2]

cleaned_data = convert_text_to_dict(input_text)

if cleaned_data:
    print(cleaned_data)


{'medicine name': {'Lubricant Eye Drops E.D.': '3 times a day for two weeks', 'Analgesic Drug TAB.': 'once after meal twice a day for a week', 'Antibiotic Eye Ointment. E.O.': 'once at night for a week'}, 'patient': 'أشرف أدهـم سـيف', 'doctor': 'معـاذ عطيتـو', 'date': '٢٤ / ۱۱ / ۲۰۲۳م'}


In [52]:
medcine = cleaned_data['medicine name']
medcine

{'Lubricant Eye Drops E.D.': '3 times a day for two weeks',
 'Analgesic Drug TAB.': 'once after meal twice a day for a week',
 'Antibiotic Eye Ointment. E.O.': 'once at night for a week'}

In [59]:
prescription_as_df = pd.DataFrame(list(medcine.items()), columns=['medicine name', 'taken number'])
prescription_as_df.head()


,medicine name,taken number
0,Lubricant Eye Drops E.D.,3 times a day for two weeks
1,Analgesic Drug TAB.,once after meal twice a day for a week
2,Antibiotic Eye Ointment. E.O.,once at night for a week


## RAG PART

In [60]:
query = "What is the side effect of doxycycline"
answer = vector_db.similarity_search(query)

In [62]:
context = "Question: " + query +\
"\n Answer: " + answer[0].page_content

In [63]:
prompt = f"""Here is the context: {context}
             Using the relevant information from the context,
             provide an answer to the query: {query}."
             If the context doesn't provide \
             any relevant information, \
             answer with \
             [I couldn't find a good match in the \
             document database for your query]
             """

In [64]:
side_effect_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=prompt,
        input_variables=["context", "query"],
    )
)
chat_prompt_template = ChatPromptTemplate.from_messages([side_effect_prompt])


chain = LLMChain(llm=llm, prompt=chat_prompt_template, verbose=True)


llm_response = chain({"context":context, "query": query})

display(llm_response)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Here is the context: Question: What is the side effect of doxycycline
 Answer: drug_name: doxycycline
side_effects: (hives, difficult breathing, swelling in your face or throat) or a severe skin reaction (fever, sore throat, burning in your eyes, skin pain, red or purple skin rash that spreads and causes blistering and peeling). Seek medical treatment if you have a serious drug reaction that can affect many parts of your body. Symptoms may include: skin rash, fever, swollen glands, flu-like symptoms, muscle aches, severe weakness, unusual bruising, or yellowing of your skin or eyes. This reaction may occur several weeks after you began using doxycycline. Doxycycline may cause serious side effects. Call your doctor at once if you have: severe stomach pain, diarrhea that is watery or bloody; throat irritation, trouble swallowing; chest pain, irregular heart rhythm, feeling short of breath; little or no urination; low whit

{'context': 'Question: What is the side effect of doxycycline\n Answer: drug_name: doxycycline\nside_effects: (hives, difficult breathing, swelling in your face or throat) or a severe skin reaction (fever, sore throat, burning in your eyes, skin pain, red or purple skin rash that spreads and causes blistering and peeling). Seek medical treatment if you have a serious drug reaction that can affect many parts of your body. Symptoms may include: skin rash, fever, swollen glands, flu-like symptoms, muscle aches, severe weakness, unusual bruising, or yellowing of your skin or eyes. This reaction may occur several weeks after you began using doxycycline. Doxycycline may cause serious side effects. Call your doctor at once if you have: severe stomach pain, diarrhea that is watery or bloody; throat irritation, trouble swallowing; chest pain, irregular heart rhythm, feeling short of breath; little or no urination; low white blood cell counts - fever, chills, swollen glands, body aches, weakness

In [65]:
display(Markdown(llm_response["text"]))

 The side effects of doxycycline include:
- Hives, difficult breathing, swelling in your face or throat, or a severe skin reaction (fever, sore throat, burning in your eyes, skin pain, red or purple skin rash that spreads and causes blistering and peeling).
- Severe stomach pain, diarrhea that is watery or bloody.
- Throat irritation, trouble swallowing.
- Chest pain, irregular heart rhythm, feeling short of breath.
- Little or no urination.
- Low white blood cell counts - fever, chills, swollen glands, body aches, weakness, pale skin, easy bruising or bleeding.
-

#using previous prescription

In [69]:
med = prescription_as_df['medicine name'].iloc[0]
query = f"What is the side effect of {med}"
answer = vector_db.similarity_search(query)
context = "Question: " + query +\
"\n Answer: " + answer[0].page_content
prompt = f"""Here is the context: {context}
             Using the relevant information from the context,
             provide an answer to the query: {query}."
             If the context doesn't provide \
             any relevant information, \
             answer with \
             [I couldn't find a good match in the \
             document database for your query]
             """
side_effect_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=prompt,
        input_variables=["context", "query"],
    )
)
chat_prompt_template = ChatPromptTemplate.from_messages([side_effect_prompt])


chain = LLMChain(llm=llm, prompt=chat_prompt_template, verbose=False)

llm_response = chain({"context":context, "query": query})

display(Markdown(llm_response["text"]))

 [I couldn't find a good match in the document database for your query]

**As the medicine is not found in the database, the generated response indicates that a suitable match couldn't be identified. This illustrates how the generated response is safeguarded against providing random or inaccurate responses.**

## TruLens

[TruLens](https://lablab.ai/tech/truera/trulens) offers a powerful suite of tools to monitor and revise the performance of LLM-based applications. Through its evaluation capabilities, TruLens enables users to assess the quality of inputs, outputs, and internal processes, incorporating built-in feedback mechanisms like groundedness, relevance, and moderation assessment, while also accommodating custom evaluation needs. It also provides essential instrumentation for a wide range of LLM applications, including question answering, retrieval-augmented generation, and agent-based solutions. This instrumentation allows users to closely monitor diverse usage metrics and metadata, offering valuable insights into the model's performance across various applications.

In [70]:
tru = Tru()
tru.reset_database()


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [72]:
class RAGMediMate:
    def __init__(self, db, llm):
        self.db = db
        self.llm = llm
        self.response = None
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = self.db.similarity_search(query)
        return results[0].page_content

    @instrument
    def generated_response(self, query: str, context: str) -> str:
        """
        Generate answer from context.
        """
        prompt = f"""Here is the context: {context}
             Using the relevant information from the context,
             provide an answer to the query: {query}."
             If the context doesn't provide \
             any relevant information, \
             answer with \
             [I couldn't find a good match in the \
             document database for your query]
             """
        side_effect_prompt = HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                template=prompt,
                input_variables=["context", "query"],
            )
        )

        chat_prompt_template = ChatPromptTemplate.from_messages([side_effect_prompt])

        chain = LLMChain(llm=self.llm, prompt=chat_prompt_template, verbose=True)

        llm_response = chain({"context":context, "query": query})
        self.response = llm_response["text"]
        return self.response

    @instrument
    def query(self, query: str) -> str:
        context = self.retrieve(query)
        res = self.generated_response(query, context)
        return res



In [74]:
rag = RAGMediMate(vector_db, llm)

In this section, we will use a feedback function that assesses both input and output, returning a numerical score. also we use Groundedness class serves as this feedback function, measuring the extent to which the output of an LLM app is grounded in the input context or knowledge.

In [84]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [85]:

litellm = LiteLLM(model_engine="chat-bison")
openai = feedback.OpenAI()
grounded = Groundedness(groundedness_provider=litellm)


# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.

f_qa_relevance = (
    Feedback(litellm.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)
qa_relevance_open_ai = Feedback(openai.relevance).on_input_output()


# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(litellm.qs_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)


✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.app.retrieve.args.query .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input statement will be set to __record__.app.retrieve.rets.collect() .


In [87]:
tru_rag = TruCustomApp(rag,
    app_id = 'MediMate',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance, qa_relevance_open_ai])

In [88]:
with tru_rag as recording:
    rag.query("What is the side effect of doxycycline?")

boto3,botocore is/are required for using BedrockEndpoint. You should be able to install it/them with
	pip install boto3 botocore


> Entering new LLMChain chain...
Prompt after formatting:
Human: Here is the context: drug_name: doxycycline
side_effects: (hives, difficult breathing, swelling in your face or throat) or a severe skin reaction (fever, sore throat, burning in your eyes, skin pain, red or purple skin rash that spreads and causes blistering and peeling). Seek medical treatment if you have a serious drug reaction that can affect many parts of your body. Symptoms may include: skin rash, fever, swollen glands, flu-like symptoms, muscle aches, severe weakness, unusual bruising, or yellowing of your skin or eyes. This reaction may occur several weeks after you began using doxycycline. Doxycycline may cause serious side effects. Call your doctor at once if you have: severe stomach pain, diarrhea that is watery or bloody; throat irritation, trouble swallowing; chest pain, irregular 

In [89]:
tru.get_leaderboard(app_ids=["MediMate"])

,Context Relevance,relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,,
MediMate,1.0,0.8,1.0,0.8,3.0,0.0


In [90]:
#run dashboard
tru.run_dashboard()

Starting dashboard ...
npx: installed 22 in 4.36s

Go to this url and submit the ip given here. your url is: https://light-ties-sort.loca.lt

  Submit this IP Address: 35.190.167.81



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [91]:
tru.stop_dashboard() # stop if needed

Dashboard closed.
Dashboard closed.
